In [ ]:
using System;
using System.IO;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Net;
using System.Threading.Tasks;
using System.Windows;

# Access modiifers
    1. Public 
    2. Private
    3. Internal
    4. Protected
    5. Protected Internal  

# Interfaces
### Why? 
Putting instances of different classes (product models) in one list (shopping cart), 
Connect to multiple Classes using same method (Composition rather than inheritance)

### Example: 
Having multiple data access for retriving and storing datas
Supports & Fittings have same method with diff operation


# Abstract Class
Fits betweeen full base class and inetrface. Blend of two.
Cannot initilize class but can be used at inheritance class
1.  Abstract method to be implement as interface on parent class - Override method on child class
2.  VIrtual - Override --> call from parent base class

### Exmaple:
Lets say we have SQL server and SQL lite to store data and have parent class with Load Connection string method as abstarct class

### When to use?


# Generics
T - Type of element in list
Can able to process multiple models of type T.
Like in Dapper to link SQL with data model.

In [ ]:
class Person{}

class LogEntry{}

public static List<T> LoadFromFile<T>(string filePath) where T : class, new()
{
    var lines = System.IO.File.ReadAllLines(filePath).ToList();
    List<T> output = new List<T>();
    T entry = new T();
    var cols = entry.GetType().GetProperties();

    //Check one header and data row
    if(lines.Count < 2){ throw new IndexOutOfRangeException(""); }

    //Split & remove headers
    var headers = lines[0].Split(",");
    lines.RemoveAt(0);

    foreach(var row in lines)
    {
        var value = row.Split(',');
        for(var i = 0; i < headers.Length; i++)
        {
            foreach(var col in cols)
            {
                if(col.Name == headers[i])
                {
                    col.SetValue(entry, Convert.ChangeType(vals[i], col.PropertyType));
                }
            } 
        }
        output.Add(entry);
    }
    return output;
}


# Events
1. Place that triggers the event
2. Palce or places that consumes the event 
3. ? used for null check before invoke
4. Remove event listeners before removing form (Memory Leakage). So don't use anonymous function fired.

### Example:
Developing Event watcher plugin in navisworks

In [ ]:
public class account
{
    public event EventHandler<string> TransApprovedEvent; 

    private List<string> _transactions = new List<string>();

    public IReadOnlyList<string> Transactions
    {
        get { return _transactions.AsReadOnly(); }
    }

    //Triggered the event
    public bool AddDeposit(string depositName)
    {
        TransApprovedEvent?.Invoke(this, depositName);
        return true;
    }
}

//Listen to event
public partial class Startwindow
{
    public Startwindow
    {
        account ac = new account();
        ac.TransApprovedEvent += account_TransApprovedEvent;
    }

    private void account_TransApprovedEvent(object sender, string s)
    {
        Customer.accounts.Update();
    }
}

# Delegates
Use Pass in method instead of properties.
Func doesnt out more than one variable so wee have to use Delegate

### Examples
Event handler is delegate 

In [ ]:
class ProductModel
{
    public decimal Prize { get; set; } 
}
public class ShoppingCartModel
{
    public delegate void MentionDiscount(decimal subTotal);

    public List<ProductModel> Items { get; set; } = new List<ProductModel>();

    public decimal GenerateTotal(MentionDiscount mentionDiscount,
        Func<List<ProductModel>, decimal, decimal> CalculateDiscountedTotal,
        Action<string> postUserDiscounting)
    {
        decimal subTotal = Items.Sum(x => x.Prize);
        mentionDiscount(subTotal);
       
        postUserDiscounting("We are applying your discount." + subTotal);

        return CalculateDiscountedTotal(Items, subTotal);
    }
}

public class wiform
{
    static ShoppingCartModel cart = new ShoppingCartModel();
    
    static void main()
    {
        Console.WriteLine($"Total is { cart.GenerateTotal(subTotalAlert, CalculateLeveledDiscount, AlertUser) }");

        decimal total = cart.GenerateTotal((subTotal) => Console.WriteLine($"Total for cart 2 is {subTotal}"),
        (products, subTotal) => {
            if (products.Count > 3)
            {
                return subTotal * 0.9m;
            }
            else    
            {
                return subTotal;
            }
        }, (message) => Console.WriteLine($"SubTotal is {message}"));
    }
    
    private static void subTotalAlert(decimal subTotal)
    {
        Console.WriteLine($"SubTotal is {subTotal}");
    }

    private static void AlertUser(string message)
    {
        Console.WriteLine(message);
    }

    private static decimal CalculateLeveledDiscount(List<ProductModel>, decimal subTotal)
    {
        if(subTotal > 100) return subTotal * 0.80M;
        else { return subTotal; }
    }
}

# ASYNC
1. Dont return from an async method - use Task if want to retuen use Task<string>
2. await - wait on something to done otherwise next steps will be fired
3. Words - async, await, Task
4. IProgress interface used to track progress of an async method with Model
5. CancellationToken - used to cancel an async method
6. Parallel Foreach - Run multiple records / Task the opration on multiple records
   


In [ ]:
private async void Click()
{
    var watch = System.Diagnostics.Stopwatch.StartNew();

    await RunDownloadAsync();

    watch.Stop();
    var elapsedMs = watch.ElapsedMilliseconds;

    Console.WriteLine($"Total execution time: { elapsedMs }");
} 

private async Task RunDownloadAsync()
{
    List<string> websites = PrepData();

    foreach (string site in websites)
    {
        WebsiteDataModel results = await Task.Run(() => DownloadWebsite(site));
        
        ReportWebsiteInfo(results);
    }
}

private async Task RunDownloadParallelAsync()
{
    List<string> websites = PrepData();
    List<Task<WebsiteDataModel>> tasks = new List<Task<WebsiteDataModel>>();

    foreach (string site in websites)
    {
        //Method 1: wrap around the task in a lambda 
        tasks.Add(Task.Run(() => DownloadWebsite(site)));
        //Method 2: able to create async method on our own
        tasks.Add(DownloadWebsiteAsync(site));
    }

    var results = await Task.WhenAll(tasks);

    foreach (var result in results)
    {
        ReportWebsiteInfo(result);
    }
}

private List<string> PrepData(){
    List<string> websites = new List<string>();
    websites.Add("http://www.google.com");
    return websites;}


private WebsiteDataModel DownloadWebsite(string websiteURL)
{
    WebsiteDataModel output = new WebsiteDataModel();
    System.Net.WebClient client = new System.Net.WebClient();

    output.WebsiteUrl = websiteURL;
    output.WebsiteData = client.DownloadString(websiteURL);

    return output;
}
private async Task<WebsiteDataModel> DownloadWebsiteAsync(string websiteURL)
{
    WebsiteDataModel output = new WebsiteDataModel();
    System.Net.WebClient client = new System.Net.WebClient();

    output.WebsiteUrl = websiteURL;
    output.WebsiteData = await client.DownloadStringTaskAsync(websiteURL);

    return output;
}
private void ReportWebsiteInfo(WebsiteDataModel data)
{
    Console.WriteLine($"{ data.WebsiteUrl } downloaded: { data.WebsiteData.Length } characters long.{ Environment.NewLine }");
}
public class WebsiteDataModel
{
    public string WebsiteUrl { get; set; } = "";
    public string WebsiteData { get; set; } = "";
}